**Direct fitting of HRP competition assay for peroxidase activity**

Chris Barry, Ché Pillay, Johann Rohwer

Redox Biochemistry and Chemistry


# Running this notebook 

This notebook can be executed with Binder, Google Colab, or locally. All the required Python packages are installed and imported.

## Binder 

If you run this notebook with Binder you don't have to install anything, Binder takes care of this.

## Google Colab  

In order to run on Google Colaboratory (Colab), a module needs to be installed and the dataset is loaded from GitHub. This is done in the first Code Cell.  
To run the notebook on Colab click on the badge:  
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Rohwer-Lab/HRP-direct-fitting/blob/main/HRP_experiment_analyser/HRP_experiment_analyser.ipynb)  
Then, in Colab, select the Code Cell below and press `Shift-Enter`.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install lmfit
    !wget https://github.com/Rohwer-Lab/HRP-direct-fitting/raw/main/HRP_experiment_analyser/HRP_data_template.xlsx
else:
    print('Not running on Colab.')

## Local environment  

If you run this notebook locally make sure you have all packages installed and the dataset (Excel template) in the same directory as the notebook. The `README` on the GitHub repository has detailed instructions for setting up a local computational environment

# Prerequisites

Paste the raw experimental data into the Excel template provided (`HRP_data_template.xlsx`) and save the file in the same directory as this notebook.

# How to run the analysis

To run the analysis perform the steps below in order.

1) Select the cell below and press `Enter` while holding `Shift`.
2) Once the cell executes,
    - Click on the `Select` button below the cell.
    - Select your Excel template file in the file browser which appears.
    - Click on the `Select` button again.  
3) Once the template has been selected,
    - Select the next cell.
    - Click on `Run` in the top menu.
    - Click `Run Selected Cell and All Below` in the drop down menu which appears.

The analysis should run to completion within several seconds.

If you want to follow the analysis step-by-step, individual code cells can be executed by selecting the cell and pressing `Shift-Enter` on the keyboard.

# How to view the results

An Excel file with the fitted rate constants and several graphs of simulations with the fitted rate constants will be created in the same location as the Excel template file.

Alternatively, the results can be viewed by scrolling down within this notebook.

In [ ]:
# Select this cell first and press `Enter` while holding `Shift`.
from ipyfilechooser import FileChooser

fc = FileChooser()
display(fc)

In [ ]:
# Once the template has been selected
#     Select this cell.
#     Click on Run in the top menu.
#     Click Run Selected Cell and All Below in the drop down menu which appears.

##### Imports

In [ ]:
import os
import copy
import math
from datetime import datetime

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from lmfit import minimize, Parameters, Parameter, report_fit
from scipy.stats import chisquare
from scipy.integrate import solve_ivp
import openpyxl
from openpyxl.styles import Font, Border, Side, PatternFill
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter

##### directories

In [ ]:
data_file_name = fc.selected_filename
data_file_path = fc.selected_path
fig_dir = data_file_path

##### data file

In [ ]:
data_file = openpyxl.load_workbook(os.path.join(data_file_path, data_file_name))
data_spreadsheet = data_file['Sheet1']

##### Matplotlib parameters

In [ ]:
%matplotlib inline

mpl_width = 4.5
mpl_height = 3.5
mpl_dpi = 600
mpl_xlabel_fontsize = 'large'
mpl_ylabel_fontsize = 'large'
mu = '\u03bc'

##### define HRP model class

In [ ]:
class HRP_model:
    """
    Class to set up the HRP assay model.
    """

    def __init__(self, k_Prx=17.0, k_HRP=17.0, H2O2_init=4, Prx_init=5, HRP_init=10, compound_I_init=0):
        self.k_Prx = k_Prx
        self.k_HRP = k_HRP
        self.H2O2_init = H2O2_init
        self.Prx_init = Prx_init
        self.HRP_init = HRP_init
        self.compound_I_init = compound_I_init

    def _dydt(self, t, y, k_HRP, k_Prx):
        # y = (HRP, Prx, H2O2, compound_I)
        v0 = k_HRP * y[0] * y[2]      # v_HRP
        v1 = k_Prx * y[1] * y[2]      # v_Prx
        dHRPdt = -v0
        dPrxdt = -v1
        dH2O2dt = -v0 - v1
        dcompound_Idt = v0
        return [dHRPdt, dPrxdt, dH2O2dt, dcompound_Idt]

    def Simulate(self, time_points=None, userinit=0):
        if userinit == 1:
            assert hasattr(self, 'sim_time'), "Time points must be specified as mod.sim_time"
            time_points = self.sim_time
        y0 = [self.HRP_init, self.Prx_init, self.H2O2_init, self.compound_I_init]
        t_span = (0, time_points.max())
        sol = solve_ivp(
            self._dydt,
            t_span,
            y0,
            t_eval=time_points,
            args=(self.k_HRP, self.k_Prx),
            atol=1e-12,
            rtol=1e-10,
            method="LSODA",
        )
        self.sol = sol
        v0 = self.k_HRP * sol.y[0] * sol.y[2]      # v_HRP
        v1 = self.k_Prx * sol.y[1] * sol.y[2]      # v_Prx
        sim = np.hstack((sol.t.reshape(len(sol.t), 1), sol.y.T, np.vstack((v0, v1)).T))
        self.sim = pd.DataFrame(
            sim, columns=['Time', 'HRP', 'Prx', 'H2O2', 'compound_I', 'v0', 'v1']
        )

##### functions

In [ ]:
def do_HRP_trace(mod, Prx, time_points):
    """
    Simulates a HRP assay trace.
    Requires "a_factor" as a GLOBAL
    """

    # Set expicit experimental concentration
    setattr(mod, 'Prx_init', Prx)

    # Do simulation
    mod.sim_time = time_points
    mod.Simulate(userinit=1)

    # Grab data
    assay_time = mod.sim['Time']
    assay_absorbance = mod.sim['compound_I'] * a_factor
    trace = np.array((assay_time, assay_absorbance)).transpose()

    return trace


def calc_f_inhi(trace, trace_HRP_only, mod):
    """
    Calculates the fractional inhibition of a HRP trace.
    """
    # Takes the mean of the last 10% of points in order to account for noise
    delta_max = trace_HRP_only[-math.floor(len(trace_HRP_only) / 10) :, 1].mean()
    delta_obs = trace[-math.floor(len(trace) / 10) :, 1].mean()
    k_HRP = getattr(mod, 'k_HRP')
    HRP_init = getattr(mod, 'HRP_init')
    f_inhi = k_HRP * HRP_init * ((delta_max - delta_obs) / delta_obs)
    return f_inhi


def calc_a_factor(e_compound_I=-5.4 * 10**4, c=10 ** (-6), l=1):
    """
    c = 10**(-6) # Analysis is in micro Molar
    l = 1 # Light path of reader
    e_compound_I = -5.4*10**4 # HRP breaks down into compound_I which is measured spectrophotometrically
    """
    a_factor = e_compound_I * c * l   # A per umol HRP (or Compound_I)
    return a_factor


def res_HRP_exp(params, mod, PSH_init_range, data_traces):
    """
    A residual function used for fitting 'k_Prx' in an HRP assay model.
    Simulates HRP assay traces under the same conditions as the data traces and finds the residual.
    """
    # Update model
    mod.k_Prx = params['k_Prx'].value

    # HRP-only trace (Prx=0)
    data_HRP_only = data_traces[0]
    trace_HRP_only = do_HRP_trace(mod, 0, time_points=data_HRP_only[:, 0])

    # Assay traces
    sim_traces = list(
        map(
            lambda PSH_init, data_trace: do_HRP_trace(
                mod, PSH_init, time_points=data_trace[:, 0]
            ),
            PSH_init_range,
            data_traces[1:],
        )
    )

    sim_traces.insert(0, trace_HRP_only)

    residuals = abs(np.vstack(sim_traces[1:]) - np.vstack(data_traces[1:]))

    # Check that time points align
    t_array_diff = residuals[:, 0] > 0.00001
    if True in t_array_diff:
        print("Warning: sim and data time arrays don't align")

    return residuals[:, 1]

##### Get user INPUT data

In [ ]:
# Name
exp_Name = data_spreadsheet['A8'].value   # - USER INPUT

# Assay parameters
exp_Prx_initials = [cell.value for cell in data_spreadsheet['27']]  # μM - USER INPUT
seen = set()
exp_Prx_initials = [x for x in exp_Prx_initials if not (x in seen or seen.add(x))]
while None in exp_Prx_initials:
    exp_Prx_initials.remove(None)
exp_Prx_initials = [float(Prx) for Prx in exp_Prx_initials]
exp_HRP_initial = float(data_spreadsheet['B14'].value)   # μM - USER INPUT
exp_H2O2_initial = float(data_spreadsheet['B15'].value)  # μM - USER INPUT

# Method parameters
exp_kHRP = float(data_spreadsheet['B16'].value)   # μM.s - USER INPUT
exp_Abs_nm = int(data_spreadsheet['B17'].value)   # only used in plot titles
e_compound_I_Abs_nm = float(
    data_spreadsheet['B18'].value
)   # -1.02*10**(5) # M-1 cm-1 e_403 
exp_lightpath = float(data_spreadsheet['B19'].value)   # cm - USER INPUT

# Data traces
trace_start_row = 28
exp_HRP_traces = []
for count, Prx in enumerate(exp_Prx_initials):
    time_col = get_column_letter(count * 2 + 1)
    time_range = data_spreadsheet[time_col]
    time_range = time_range[trace_start_row:]

    Abs_col = get_column_letter(count * 2 + 2)
    Abs_range = data_spreadsheet[Abs_col]
    Abs_range = Abs_range[trace_start_row:]

    trace = list()
    for time, Abs in zip(time_range, Abs_range):
        if time.value is None or Abs.value is None:
            break
        else:
            trace.append([time.value, Abs.value])
    trace = np.array(trace)
    exp_HRP_traces.append(trace)

##### calculate absorbance factor

In [ ]:
a_factor = calc_a_factor(e_compound_I=e_compound_I_Abs_nm, l=exp_lightpath)

## Fit k_Prx by ODE

### Setup model

In [ ]:
mod_HRP_Prx = HRP_model()

setattr(mod_HRP_Prx, 'HRP_init', exp_HRP_initial)
setattr(mod_HRP_Prx, 'H2O2_init', exp_H2O2_initial)
setattr(mod_HRP_Prx, 'k_HRP', exp_kHRP)

### Fit to each trace

In [ ]:
mod_direct_indiv = copy.deepcopy(mod_HRP_Prx)

In [ ]:
# Fit
direct_fit_indiv_kPrxs = []
direct_fit_indiv_stderr = []

for count, exp_HRP_trace in enumerate(exp_HRP_traces[1:]):
    param_lib = Parameters()
    param_lib.add('k_Prx', value=1, min=0.0)
    fit_kPrx = minimize(
        res_HRP_exp,
        param_lib,
        args=(
            mod_direct_indiv,
            [exp_Prx_initials[1:][count]],
            [exp_HRP_traces[0], exp_HRP_trace],
        ),
    )
    direct_fit_indiv_kPrxs.append(fit_kPrx.params['k_Prx'].value)
    direct_fit_indiv_stderr.append(fit_kPrx.params['k_Prx'].stderr)

In [ ]:
# Display fitted values
print(direct_fit_indiv_kPrxs)

In [ ]:
sim_HRP_traces = []

for count, kPrx in enumerate(direct_fit_indiv_kPrxs):
    setattr(mod_direct_indiv, 'k_Prx', kPrx)

    HRP_trace = do_HRP_trace(
        mod_direct_indiv,
        exp_Prx_initials[1:][count],
        time_points=exp_HRP_traces[1:][count][:, 0],
    )

    sim_HRP_traces.append(HRP_trace)

In [ ]:
# Plot analysis

# Assign data
HRP_traces_exp = exp_HRP_traces
HRP_traces_sim = sim_HRP_traces

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(HRP_traces_exp[0][:, 0], HRP_traces_exp[0][:, 1], color='dimgrey', label='0')
for count, trace in enumerate(HRP_traces_exp[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], label=f'{exp_Prx_initials[1:][count]}')

axarr.plot(HRP_traces_sim[0][:, 0], HRP_traces_sim[0][:, 1], 'k--')
for count, trace in enumerate(HRP_traces_sim[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], 'k--')

axarr.set_xlabel('Time (s)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)
axarr.legend(title=f'Prx ({mu}M)', ncols=3, columnspacing=1, loc='upper right')

f.tight_layout()

f.savefig(
    os.path.join(fig_dir, f'HRP_direct_ind-fit_traces_{exp_Name}.pdf'),
    dpi=mpl_dpi,
)

# chi square
chi_square_dir_indiv = sum(
    (np.vstack(HRP_traces_sim)[:,1]-np.vstack(HRP_traces_exp[1:])[:,1])**2
)
print(chi_square_dir_indiv)

### Fit to all traces simultaneously

In [ ]:
mod_direct_all = copy.deepcopy(mod_HRP_Prx)

In [ ]:
param_lib = Parameters()
param_lib.add('k_Prx', value=1, min=0.0)
fit = minimize(
    res_HRP_exp, param_lib, args=(mod_direct_all, exp_Prx_initials[1:], exp_HRP_traces)
)
direct_fit_whole_kPrx = fit.params['k_Prx'].value
direct_fit_whole_stderr = fit.params['k_Prx'].stderr

In [ ]:
report_fit(fit)

In [ ]:
# Simulate traces with fitted k_Prx
setattr(mod_direct_all, 'k_Prx', direct_fit_whole_kPrx)

sim_HRP_traces = []

for count, SH_init in enumerate(exp_Prx_initials[1:]):

    HRP_trace = do_HRP_trace(
        mod_direct_all,
        exp_Prx_initials[1:][count],
        time_points=exp_HRP_traces[1:][count][:, 0],
    )

    sim_HRP_traces.append(HRP_trace)

In [ ]:
# Plot analysis

# Assign data
HRP_traces_exp = exp_HRP_traces
HRP_traces_sim = sim_HRP_traces

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(HRP_traces_exp[0][:, 0], HRP_traces_exp[0][:, 1], color='dimgrey', label='0')
for count, trace in enumerate(HRP_traces_exp[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], label=f'{exp_Prx_initials[1:][count]}')

axarr.plot(HRP_traces_sim[0][:, 0], HRP_traces_sim[0][:, 1], 'k--')
for count, trace in enumerate(HRP_traces_sim[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], 'k--')

axarr.set_xlabel('Time (s)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)
axarr.legend(title=f'Prx ({mu}M)', ncols=3, columnspacing=1, loc='upper right')

f.tight_layout()

f.savefig(
    os.path.join(data_file_path, f'HRP_direct_wh-fit_traces_{exp_Name}.pdf'),
    dpi=mpl_dpi,
)

# chi square
chi_square_dir_indiv = sum(
    (np.vstack(HRP_traces_sim)[:,1]-np.vstack(HRP_traces_exp[1:])[:,1])**2
)
print(chi_square_dir_indiv)

## Calculate k_Prx by fractional inhibition

### Fit to all traces simultaneously

In [ ]:
mod_finhi_all = copy.deepcopy(mod_HRP_Prx)

In [ ]:
# Plot analysis

# Assign data
HRP_traces = exp_HRP_traces

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(HRP_traces[0][:, 0], HRP_traces[0][:, 1], color='dimgrey', label='0')

for count, trace in enumerate(HRP_traces[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], label=f'{exp_Prx_initials[1:][count]}')

axarr.set_xlabel('Time (s)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)
axarr.legend(title=f'Prx ({mu}M)', ncols=3, columnspacing=1, loc='upper right')

f.tight_layout()

# f.savefig(os.path.join(fig_dir,f"HRP_traces_{exp_Name}.pdf"),dpi=mpl_dpi)

In [ ]:
# Plot analysis

# Assign data
HRP_traces = exp_HRP_traces

dAbs = list(
    abs(trace[-math.floor(len(trace) / 10) :, 1].mean()) for trace in HRP_traces
)

HRP_dAbs = np.array(list(zip(exp_Prx_initials, dAbs)))

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

bar_width = (
    HRP_dAbs[:, 0].max() / len(HRP_dAbs[:, 0]) * 0.2
)   # set bar_width to a fraction of space per data point

plt.bar(HRP_dAbs[:, 0], HRP_dAbs[:, 1], width=bar_width, align='center')

axarr.set_xticks(HRP_dAbs[:, 0])
axarr.set_xlabel(f'Prx ({mu}M)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'$\Delta$ Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)

f.tight_layout()

f.savefig(os.path.join(fig_dir, f'HRP_f-inhi_dAbs_{exp_Name}.pdf'), dpi=mpl_dpi)

In [ ]:
# Plot analysis
HRP_traces = exp_HRP_traces

# Assign data
HRP_f_inhi = list(
    map(lambda trace: calc_f_inhi(trace, HRP_traces[0], mod_finhi_all), HRP_traces[1:])
)
HRP_f_inhi = np.array((exp_Prx_initials[1:], HRP_f_inhi)).transpose()
HRP_f_inhi_lm = LinearRegression().fit(
    HRP_f_inhi[:, 0].reshape(-1, 1), HRP_f_inhi[:, 1]
)
frac_inhi_whole_kPrx = HRP_f_inhi_lm.coef_[0]

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(
    HRP_f_inhi[:, 0],
    HRP_f_inhi[:, 1],
    'bo',
    label=f'$k_{{Prx}} = {frac_inhi_whole_kPrx:.3}\ \mathrm{{ µM^{{{-1}}}\cdot s^{{{-1}}} }}$',
)
axarr.plot(
    np.hstack(([0], HRP_f_inhi[:, 0])),
    HRP_f_inhi_lm.predict(np.hstack(([0], HRP_f_inhi[:, 0])).reshape(-1, 1)),
    f'k-',
)

axarr.set_xlabel(f'[Prx] ({mu}M)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(
    '$k\mathregular{{_{{HRP}}}}$$\cdot$[HRP]$\cdot$\
($\Delta A_{{{max}}}-\Delta A_{{{obs}}}$)/$\Delta A_{{{obs}}}$',
    fontsize=mpl_ylabel_fontsize,
)

axarr.set_ylim(bottom=0)
axarr.set_xlim(left=0)
axarr.legend()

f.tight_layout()

f.savefig(os.path.join(fig_dir, f'HRP_f-inhi_vs_Prx_{exp_Name}.pdf'), dpi=mpl_dpi)

In [ ]:
setattr(mod_finhi_all, 'k_Prx', frac_inhi_whole_kPrx)

sim_HRP_traces = []

for count, SH_init in enumerate(exp_Prx_initials[1:]):

    HRP_trace = do_HRP_trace(
        mod_finhi_all,
        exp_Prx_initials[1:][count],
        time_points=exp_HRP_traces[1:][count][:, 0],
    )

    sim_HRP_traces.append(HRP_trace)

In [ ]:
# Plot analysis

# Assign data
HRP_traces_exp = exp_HRP_traces
HRP_traces_sim = sim_HRP_traces

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(HRP_traces_exp[0][:, 0], HRP_traces_exp[0][:, 1], color='dimgrey', label='0')
for count, trace in enumerate(HRP_traces_exp[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], label=f'{exp_Prx_initials[1:][count]}')

axarr.plot(HRP_traces_sim[0][:, 0], HRP_traces_sim[0][:, 1], 'k--')
for count, trace in enumerate(HRP_traces_sim[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], 'k--')

axarr.set_xlabel('Time (s)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)
axarr.legend(title=f'Prx ({mu}M)', ncols=3, columnspacing=1, loc='upper right')

f.tight_layout()

f.savefig(
    os.path.join(fig_dir, f'HRP_f-inhi_wh-fit_traces_{exp_Name}.pdf'),
    dpi=mpl_dpi,
)

# chi square
chi_square_dir_indiv = sum(
    (np.vstack(HRP_traces_sim)[:,1]-np.vstack(HRP_traces_exp[1:])[:,1])**2
)
print(chi_square_dir_indiv)

### Fit to each trace

In [ ]:
mod_finhi_indiv = copy.deepcopy(mod_HRP_Prx)

In [ ]:
frac_inhi_indiv_kPrxs = HRP_f_inhi[:, 1] / HRP_f_inhi[:, 0] - HRP_f_inhi_lm.intercept_
print(f'fitted k_Prxs: {frac_inhi_indiv_kPrxs}')

In [ ]:
# Simulate each trace with k_Prx fitted to that trace
sim_HRP_traces = []

for count, kPrx in enumerate(frac_inhi_indiv_kPrxs):
    setattr(mod_finhi_indiv, 'k_Prx', kPrx)

    HRP_trace = do_HRP_trace(
        mod_finhi_indiv,
        exp_Prx_initials[1:][count],
        time_points=exp_HRP_traces[1:][count][:, 0],
    )

    sim_HRP_traces.append(HRP_trace)

In [ ]:
# Plot analysis

# Assign data
HRP_traces_exp = exp_HRP_traces
HRP_traces_sim = sim_HRP_traces

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(HRP_traces_exp[0][:, 0], HRP_traces_exp[0][:, 1], color='dimgrey', label='0')
for count, trace in enumerate(HRP_traces_exp[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], label=f'{exp_Prx_initials[1:][count]}')

axarr.plot(HRP_traces_sim[0][:, 0], HRP_traces_sim[0][:, 1], 'k--')
for count, trace in enumerate(HRP_traces_sim[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], 'k--')

axarr.set_xlabel('Time (s)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)
axarr.legend(title=f'Prx ({mu}M)', ncols=3, columnspacing=1, loc='upper right')

f.tight_layout()

f.savefig(
    os.path.join(fig_dir, f'HRP_f-inhi_ind-fit_traces_{exp_Name}.pdf'),
    dpi=mpl_dpi,
)

# chi square
chi_square_dir_indiv = sum(
    (np.vstack(HRP_traces_sim)[:,1]-np.vstack(HRP_traces_exp[1:])[:,1])**2
)
print(chi_square_dir_indiv)

## Output to Excel

In [ ]:
# Styles
heading_1 = Font(name='Calibri', size=15, bold=True)
heading_2 = Font(name='Calibri', bold=True)
shade_1 = PatternFill(start_color='00C0C0C0', end_color='00C0C0C0', fill_type='solid')
border_1 = Side(border_style='thin')
num_1 = '0.00'
num_2 = '0.000'


def apply_bottom_border(cells, ws, border_style=border_1):

    for cell in cells:
        ws[cell].border = Border(bottom=border_style)


neg_one_super = '\u207B\u00B9'
cdot = '\u00B7'

In [ ]:
# Write
wb = openpyxl.Workbook()
ws = wb.active

# Header
ws.append(['Analysis of HRP direct fit data'])
ws.append([])
ws.append(['Exp. name', exp_Name])
ws.append(['Data filename', data_file_name])
ws['A1'].font = heading_1
ws['A3'].font = heading_2
ws['A4'].font = heading_2

# Date and time
ws.append(['Date', datetime.now().date()])
ws['A5'].font = heading_2
ws.append(['Time', datetime.now().time()])
ws['A6'].font = heading_2

ws.append([])

# Direct fitting rate constants
rc = 8
ws.append(['Direct fitting'])
ws[f'A{rc}'].font = heading_1
apply_bottom_border([f'A{rc}', f'B{rc}', f'C{rc}'], ws)

rc += 1
ws.append(['Trace', f'k (µM{neg_one_super}{cdot}s{neg_one_super})', 'stderr'])
ws[f'A{rc}'].font = heading_2
ws[f'B{rc}'].font = heading_2
ws[f'C{rc}'].font = heading_2
apply_bottom_border([f'A{rc}', f'B{rc}', f'C{rc}'], ws)

for count, Prx in enumerate(exp_Prx_initials[1:]):
    rc += 1
    ws.append(
        [f'{Prx} µM', direct_fit_indiv_kPrxs[count], direct_fit_indiv_stderr[count]]
    )
    ws[f'B{rc}'].number_format = num_1
    ws[f'C{rc}'].number_format = num_2

rc += 1
ws.append(['All', direct_fit_whole_kPrx, direct_fit_whole_stderr])
ws[f'A{rc}'].font = heading_2
ws[f'B{rc}'].fill = shade_1
ws[f'C{rc}'].fill = shade_1
ws[f'B{rc}'].number_format = num_1
ws[f'C{rc}'].number_format = num_2
apply_bottom_border([f'A{rc}', f'B{rc}', f'C{rc}'], ws)

rc += 1
ws.append([])

# Fractional inhibition rate constants
rc += 1
ws.append(['Fractional inhibition'])
ws[f'A{rc}'].font = heading_1
apply_bottom_border([f'A{rc}', f'B{rc}'], ws)

rc += 1
ws.append(['Trace', f'k (µM{neg_one_super}{cdot}s{neg_one_super})'])
ws[f'A{rc}'].font = heading_2
ws[f'B{rc}'].font = heading_2
apply_bottom_border([f'A{rc}', f'B{rc}'], ws)

for count, Prx in enumerate(exp_Prx_initials[1:]):
    rc += 1
    ws.append([f'{Prx} µM', frac_inhi_indiv_kPrxs[count]])
    ws[f'B{rc}'].number_format = num_1

rc += 1
ws.append(['All', frac_inhi_whole_kPrx])
ws[f'A{rc}'].font = heading_2
ws[f'B{rc}'].number_format = num_1
ws[f'B{rc}'].fill = shade_1
apply_bottom_border([f'A{rc}', f'B{rc}'], ws)

dim_holder = DimensionHolder(worksheet=ws)

for col in range(ws.min_column, ws.max_column + 1):
    dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=14)

ws.column_dimensions = dim_holder

wb.save(os.path.join(data_file_path, f'Analysis_of_{exp_Name}.xlsx'))